## Bài 1

In [6]:
# ===============================================================
# BÀI 1. Vấn đề Cold-Start trong Hệ thống Gợi ý (Recommender System)
# Tác giả: Trần Chung Hiếu
# ===============================================================

import pandas as pd
import numpy as np

# ---------------------------------------------------------------
# 🧠 1️⃣ Đọc file dữ liệu CSV
# File gồm: user_id, song_id, rating
# ---------------------------------------------------------------

# Đọc file (giả sử tên file là "songs.csv")
df = pd.read_csv("data.csv")

print("=== DỮ LIỆU BAN ĐẦU ===")
print(df)
print()

# ---------------------------------------------------------------
# 🧩 2️⃣ Cold-start user (user mới, chưa có lịch sử nghe)
#    → Dựa vào số lượt rating và rating trung bình
# ---------------------------------------------------------------

# Tính số lượng rating và trung bình rating cho mỗi bài hát
song_stats = df.groupby('song_id')['rating'].agg(['count', 'mean']).reset_index()

# Tính trung bình rating toàn hệ thống
global_mean = df['rating'].mean()

# Hệ số điều chỉnh để tránh ưu tiên các bài có ít rating
k = 5

# Tính điểm tổng hợp theo công thức Bayesian Average
song_stats['score'] = (song_stats['count'] / (song_stats['count'] + k)) * song_stats['mean'] \
                      + (k / (song_stats['count'] + k)) * global_mean

# Sắp xếp giảm dần theo điểm tổng hợp
top5_cold_start = song_stats.sort_values('score', ascending=False).head(5)

print("=== 🧠 GỢI Ý CHO USER MỚI (COLD-START USER) ===")
print(top5_cold_start[['song_id', 'count', 'mean', 'score']])
print()

# ---------------------------------------------------------------
# 🎧 3️⃣ Gợi ý cho user đã nghe bài hát STT = 1
#    → Dùng kỹ thuật Co-occurrence (đồng xuất hiện)
# ---------------------------------------------------------------

# Tạo ma trận user-song: mỗi ô = 1 nếu user nghe bài hát đó
pivot = pd.crosstab(df['user_id'], df['song_id'])

# Tính số lần 2 bài hát được nghe cùng nhau
co_occurrence = pivot.T.dot(pivot)

# Xóa chính nó (để không tự so sánh chính mình)
np.fill_diagonal(co_occurrence.values, 0)

# Lấy hàng tương ứng với bài hát STT = 1 (nếu tồn tại)
if 1 in co_occurrence.index:
    song1_related = co_occurrence.loc[1].sort_values(ascending=False).head(5)
    print("=== 🎧 GỢI Ý CHO USER ĐÃ NGHE BÀI HÁT STT = 1 ===")
    print(song1_related)
else:
    print("⚠️ Bài hát STT = 1 chưa xuất hiện trong dữ liệu, không thể tính co-occurrence.")
print()

# ---------------------------------------------------------------
# 🆕 4️⃣ Nếu có bài hát mới thêm vào hệ thống
# ---------------------------------------------------------------

print("=== 🆕 KHI CÓ ITEM (BÀI HÁT) MỚI THÊM VÀO ===")
print("""
Trường hợp này là 'Item Cold-Start Problem':
→ Giải pháp xử lý:
1️⃣ Dùng Content-based Filtering: dựa vào thể loại, ca sĩ, tempo, cảm xúc, v.v.
2️⃣ Dùng Hybrid model: kết hợp giữa dữ liệu nội dung và collaborative filtering.
3️⃣ Thu thập một vài lượt nghe hoặc rating ban đầu (warm-start).
4️⃣ Nếu chưa có dữ liệu gì: tạm đề xuất theo độ phổ biến của thể loại tương tự.
""")


=== DỮ LIỆU BAN ĐẦU ===
     user_id  song_id  rating
0          1       18       5
1          3       12       4
2          3        9       3
3          3       15       1
4          3       12       2
..       ...      ...     ...
437      100       17       3
438      100        7       5
439      100       16       2
440      100       11       3
441      100       15       5

[442 rows x 3 columns]

=== 🧠 GỢI Ý CHO USER MỚI (COLD-START USER) ===
    song_id  count      mean     score
17       18     18  3.555556  3.435766
6         7     14  3.571429  3.422243
9        10     16  3.500000  3.382030
14       15     31  3.354839  3.306184
3         4     19  3.368421  3.292609

=== 🎧 GỢI Ý CHO USER ĐÃ NGHE BÀI HÁT STT = 1 ===
song_id
12    18
14    11
11    10
15     9
17     9
Name: 1, dtype: int64

=== 🆕 KHI CÓ ITEM (BÀI HÁT) MỚI THÊM VÀO ===

Trường hợp này là 'Item Cold-Start Problem':
→ Giải pháp xử lý:
1️⃣ Dùng Content-based Filtering: dựa vào thể loại, ca sĩ, tempo, cảm xúc,

## Bài 2

In [8]:
# ============================================================
# BÀI 2. Vấn đề xây dựng Content-Based Data (MovieLens)
# Tác giả: Trần Chung Hiếu
# ============================================================

import pandas as pd
from itertools import combinations

# ------------------------------------------------------------
# 🧠 1️⃣ Đọc file dữ liệu movies.csv
# ------------------------------------------------------------
df = pd.read_csv("./movie_lens/movies.csv")

print("=== DỮ LIỆU GỐC ===")
print(df.head(), "\n")

# ------------------------------------------------------------
# 🧩 2️⃣ Tách các thể loại thành danh sách riêng biệt
# ------------------------------------------------------------
df['genre_list'] = df['genres'].apply(lambda x: x.split('|') if isinstance(x, str) else [])

# Thêm cột đếm số thể loại
df['genre_count'] = df['genre_list'].apply(len)

# ------------------------------------------------------------
# 🎬 3️⃣ Lọc 100 phim có nhiều thể loại nhất
# ------------------------------------------------------------
top100 = df.sort_values('genre_count', ascending=False).head(100).reset_index(drop=True)

print("=== 100 phim có nhiều thể loại nhất ===")
print(top100[['movieId', 'title', 'genre_count']].head(10), "\n")

# ------------------------------------------------------------
# 🎞️ 4️⃣ Tính độ tương đồng Jaccard giữa các phim
# ------------------------------------------------------------

def jaccard_similarity(setA, setB):
    """Tính độ tương đồng Jaccard giữa 2 tập thể loại"""
    if not setA or not setB:
        return 0
    return len(setA & setB) / len(setA | setB)

# ------------------------------------------------------------
# 🧩 5️⃣ Chọn một phim X (ví dụ: movieId = 1) và tìm top 10 phim tương đồng
# ------------------------------------------------------------
film_X = top100[top100['movieId'] == 1].iloc[0]
genres_X = set(film_X['genre_list'])

similarities = []
for idx, row in top100.iterrows():
    if row['movieId'] == film_X['movieId']:
        continue
    sim = jaccard_similarity(genres_X, set(row['genre_list']))
    similarities.append((row['movieId'], row['title'], sim))

# Sắp xếp theo độ tương đồng giảm dần
similarities = sorted(similarities, key=lambda x: x[2], reverse=True)
top10_similar = similarities[:10]

print(f"=== 🎞️ Top 10 phim tương đồng nhất với: {film_X['title']} ===")
for movieId, title, sim in top10_similar:
    print(f"{title:50s}  →  Jaccard = {sim:.2f}")
print()

# ------------------------------------------------------------
# 👤 6️⃣ Xây dựng hồ sơ nội dung (User Profile) cho user Y
#     User Y đã xem các phim 1, 2, 3, 4, 5
# ------------------------------------------------------------
user_watched_ids = [1, 2, 3, 4, 5]

# Lấy danh sách thể loại từ các phim user đã xem
watched_genres = top100[top100['movieId'].isin(user_watched_ids)]['genre_list']

# Gộp tất cả thể loại vào 1 tập hợp duy nhất
user_profile = set([genre for sublist in watched_genres for genre in sublist])

print("=== 👤 Hồ sơ nội dung (User Profile) ===")
print(user_profile, "\n")

# ------------------------------------------------------------
# 🎯 7️⃣ Tính độ tương đồng giữa profile và các phim còn lại
# ------------------------------------------------------------
recommendations = []
for idx, row in top100.iterrows():
    if row['movieId'] in user_watched_ids:
        continue
    sim = jaccard_similarity(user_profile, set(row['genre_list']))
    recommendations.append((row['movieId'], row['title'], sim))

# Sắp xếp và lấy top 10 gợi ý
recommendations = sorted(recommendations, key=lambda x: x[2], reverse=True)[:10]

print("=== 🎯 Top 10 phim gợi ý cho user Y ===")
for movieId, title, sim in recommendations:
    print(f"{title:50s}  →  Jaccard(User, Film) = {sim:.2f}")


=== DỮ LIỆU GỐC ===
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy   

=== 100 phim có nhiều thể loại nhất ===
   movieId                                              title  genre_count
0    81132                                      Rubber (2010)           10
1    26701  Patlabor: The Movie (Kidô keisatsu patorebâ: T...            8
2    71999          Aelita: The Queen of Mars (Aelita) (1924)            7
3     1907                                 

## Bài 3

In [9]:
import pandas as pd
import numpy as np

# ------------------------------
# 1️⃣ Tạo utility matrix
# ------------------------------
ratings = pd.DataFrame({
    'I1': [4, 3, 5, np.nan],
    'I2': [3, 3, 4, 2],
    'I3': [np.nan, 5, 3, 3],
    'I4': [4, 5, 4, 5]
}, index=['U1', 'U2', 'U3', 'U4'])

print("=== Utility Matrix (ban đầu) ===")
print(ratings, "\n")

# ------------------------------
# 2️⃣ Tính trung bình mỗi user (dùng cho Pearson)
# ------------------------------
user_mean = ratings.mean(axis=1)

# ------------------------------
# 3️⃣ Hàm tính Pearson similarity giữa 2 user
# ------------------------------
def pearson(u1, u2):
    common = ratings.loc[[u1, u2]].dropna(axis=1)
    if common.shape[1] < 2:  # cần ít nhất 2 điểm chung
        return 0
    a = common.loc[u1] - common.loc[u1].mean()
    b = common.loc[u2] - common.loc[u2].mean()
    num = np.sum(a * b)
    den = np.sqrt(np.sum(a**2) * np.sum(b**2))
    return num / den if den != 0 else 0

# ------------------------------
# 4️⃣ Tính toàn bộ ma trận tương đồng Pearson
# ------------------------------
users = ratings.index
sim = pd.DataFrame(np.zeros((len(users), len(users))), index=users, columns=users)
for u1 in users:
    for u2 in users:
        if u1 != u2:
            sim.loc[u1, u2] = pearson(u1, u2)

print("=== Ma trận tương đồng Pearson ===")
print(sim.round(2), "\n")

# ------------------------------
# 5️⃣ Hàm dự đoán rating bằng Pearson
# ------------------------------
def predict_pearson(target_user, target_item):
    sims = []
    weighted = []
    for other in users:
        if other == target_user:
            continue
        if pd.notna(ratings.loc[other, target_item]):
            s = sim.loc[target_user, other]
            if s != 0:
                sims.append(abs(s))
                weighted.append(s * (ratings.loc[other, target_item] - user_mean[other]))
    if len(sims) == 0:
        return user_mean[target_user]
    return user_mean[target_user] + sum(weighted) / sum(sims)

# Dự đoán 2 giá trị missing
r_U1_I3 = predict_pearson('U1', 'I3')
r_U4_I1 = predict_pearson('U4', 'I1')

print(f"🎯 Dự đoán Pearson:")
print(f"r(U1, I3) = {r_U1_I3:.2f}")
print(f"r(U4, I1) = {r_U4_I1:.2f}\n")


=== Utility Matrix (ban đầu) ===
     I1  I2   I3  I4
U1  4.0   3  NaN   4
U2  3.0   3  5.0   5
U3  5.0   4  3.0   4
U4  NaN   2  3.0   5 

=== Ma trận tương đồng Pearson ===
     U1    U2    U3    U4
U1  0.0  0.50  0.50  1.00
U2  0.5  0.00 -0.71  0.76
U3  0.5 -0.71  0.00  0.19
U4  1.0  0.76  0.19  0.00 

🎯 Dự đoán Pearson:
r(U1, I3) = 3.50
r(U4, I1) = 3.21



##  Bài 4

In [10]:
# ==========================================================
# BÀI 4. Lọc cộng tác User-based & Item-based
# Tác giả: Trần Chung Hiếu
# ==========================================================

import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# ----------------------------------------------------------
# 🧠 1️⃣ Utility matrix (giống bài 3)
# ----------------------------------------------------------
ratings = pd.DataFrame({
    'I1': [4, 3, 5, np.nan],
    'I2': [3, 3, 4, 2],
    'I3': [np.nan, 5, 3, 3],
    'I4': [4, 5, 4, 5]
}, index=['U1', 'U2', 'U3', 'U4'])

print("=== Utility Matrix (rating ban đầu) ===")
print(ratings, "\n")

# ----------------------------------------------------------
# 🧩 2️⃣ Tính ma trận tương đồng giữa các user (User-based)
# ----------------------------------------------------------
# Điền missing bằng 0 tạm thời để tính cosine
ratings_filled = ratings.fillna(0)

# Tính similarity giữa các user
user_sim = cosine_similarity(ratings_filled)
user_sim_df = pd.DataFrame(user_sim, index=ratings.index, columns=ratings.index)

print("=== Ma trận tương đồng giữa các USER ===")
print(user_sim_df.round(2), "\n")

# ----------------------------------------------------------
# 🎬 3️⃣ Tính ma trận tương đồng giữa các item (Item-based)
# ----------------------------------------------------------
item_sim = cosine_similarity(ratings_filled.T)
item_sim_df = pd.DataFrame(item_sim, index=ratings.columns, columns=ratings.columns)

print("=== Ma trận tương đồng giữa các ITEM ===")
print(item_sim_df.round(2), "\n")

# ----------------------------------------------------------
# 👤 4️⃣ USER-BASED RECOMMENDATION
# ----------------------------------------------------------
def recommend_for_user(user_id, top_k=1):
    """Gợi ý item cho 1 user dựa trên user-based CF"""
    user_idx = ratings.index.get_loc(user_id)
    sim_scores = user_sim[user_idx]            # độ tương đồng của user hiện tại với các user khác
    rated_items = ratings.loc[user_id].dropna().index  # các item user đã rate
    preds = {}

    for item in ratings.columns:
        if item in rated_items:
            continue
        numer, denom = 0, 0
        for other_idx, other_user in enumerate(ratings.index):
            if other_user == user_id:
                continue
            rating = ratings.loc[other_user, item]
            if not np.isnan(rating):
                numer += sim_scores[other_idx] * rating
                denom += abs(sim_scores[other_idx])
        preds[item] = numer / denom if denom != 0 else 0

    # Sắp xếp gợi ý
    sorted_preds = sorted(preds.items(), key=lambda x: x[1], reverse=True)
    return sorted_preds[:top_k]

print("=== 👤 Gợi ý item cho từng user (User-based CF) ===")
for user in ratings.index:
    recs = recommend_for_user(user, top_k=2)
    print(f"{user}: {recs}")
print()

# ----------------------------------------------------------
# 🎞️ 5️⃣ ITEM-BASED RECOMMENDATION
# ----------------------------------------------------------
def recommend_for_item(item_id, top_k=1):
    """Gợi ý user nào có thể thích item này (item-based CF)"""
    item_idx = ratings.columns.get_loc(item_id)
    sim_scores = item_sim[item_idx]            # độ tương đồng của item với các item khác
    preds = {}

    for user in ratings.index:
        if not np.isnan(ratings.loc[user, item_id]):
            continue  # user đã rate rồi
        numer, denom = 0, 0
        for other_item in ratings.columns:
            rating = ratings.loc[user, other_item]
            if not np.isnan(rating):
                other_idx = ratings.columns.get_loc(other_item)
                numer += sim_scores[other_idx] * rating
                denom += abs(sim_scores[other_idx])
        preds[user] = numer / denom if denom != 0 else 0

    sorted_preds = sorted(preds.items(), key=lambda x: x[1], reverse=True)
    return sorted_preds[:top_k]

print("=== 🎞️ Gợi ý user nào có thể thích từng item (Item-based CF) ===")
for item in ratings.columns:
    recs = recommend_for_item(item, top_k=2)
    print(f"{item}: {recs}")


=== Utility Matrix (rating ban đầu) ===
     I1  I2   I3  I4
U1  4.0   3  NaN   4
U2  3.0   3  5.0   5
U3  5.0   4  3.0   4
U4  NaN   2  3.0   5 

=== Ma trận tương đồng giữa các USER ===
      U1    U2    U3    U4
U1  1.00  0.78  0.92  0.66
U2  0.78  1.00  0.93  0.90
U3  0.92  0.93  1.00  0.74
U4  0.66  0.90  0.74  1.00 

=== Ma trận tương đồng giữa các ITEM ===
      I1    I2    I3    I4
I1  1.00  0.94  0.65  0.80
I2  0.94  1.00  0.82  0.95
I3  0.65  0.82  1.00  0.88
I4  0.80  0.95  0.88  1.00 

=== 👤 Gợi ý item cho từng user (User-based CF) ===
U1: [('I3', np.float64(3.6586220543409183))]
U2: []
U3: []
U4: [('I1', np.float64(3.9278575235231488))]

=== 🎞️ Gợi ý user nào có thể thích từng item (Item-based CF) ===
I1: [('U4', np.float64(3.273634213464848))]
I2: []
I3: [('U1', np.float64(3.6509863183043927))]
I4: []
